In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("merged_tennis_files/merged_file.csv")

# Required columns for prediction
required_cols = [
    'winner_name', 'loser_name',
    'winner_age', 'loser_age',
    'winner_rank', 'loser_rank',
    'winner_ht', 'loser_ht',
    'minutes', 'surface'
]

df = df.dropna(subset=required_cols).reset_index(drop=True)


In [2]:
df['tourney_date'] = pd.to_datetime(df['tourney_date'], format='%Y%m%d')
df['surface'] = df['surface'].str.upper()
df = df[df['score'].notna()]  # remove walkovers


In [3]:
mask = np.random.rand(len(df)) > 0.5


In [4]:
ml_df = pd.DataFrame()

# Match context
ml_df['surface'] = df['surface']
ml_df['best_of'] = df['best_of']
ml_df['round'] = df['round']
ml_df['minutes'] = df['minutes']

# Player A
ml_df['player_a_name'] = np.where(mask, df['loser_name'], df['winner_name'])
ml_df['player_a_age'] = np.where(mask, df['loser_age'], df['winner_age'])
ml_df['player_a_rank'] = np.where(mask, df['loser_rank'], df['winner_rank'])
ml_df['player_a_height'] = np.where(mask, df['loser_ht'], df['winner_ht'])
ml_df['player_a_hand'] = np.where(mask, df['loser_hand'], df['winner_hand'])

# Player B
ml_df['player_b_name'] = np.where(mask, df['winner_name'], df['loser_name'])
ml_df['player_b_age'] = np.where(mask, df['winner_age'], df['loser_age'])
ml_df['player_b_rank'] = np.where(mask, df['winner_rank'], df['loser_rank'])
ml_df['player_b_height'] = np.where(mask, df['winner_ht'], df['loser_ht'])
ml_df['player_b_hand'] = np.where(mask, df['winner_hand'], df['loser_hand'])

# Difference features
ml_df['rank_diff'] = ml_df['player_a_rank'] - ml_df['player_b_rank']
ml_df['age_diff'] = ml_df['player_a_age'] - ml_df['player_b_age']
ml_df['height_diff'] = ml_df['player_a_height'] - ml_df['player_b_height']

# Target
ml_df['player_a_win'] = np.where(mask, 0, 1)


In [5]:
ml_df = ml_df.dropna().reset_index(drop=True)


In [6]:
ml_df.to_csv("tennis_matches_ml_with_names.csv", index=False)
